<a href="https://colab.research.google.com/github/aaaksenova/NLP_ABSA_project/blob/change/tonal_dict_syntax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy==3.2.0
!python -m spacy download ru_core_news_lg

     |████████████████████████████████| 6.0 MB 4.6 MB/s 
     |████████████████████████████████| 181 kB 55.5 MB/s 
     |████████████████████████████████| 10.1 MB 35.8 MB/s 
     |████████████████████████████████| 628 kB 45.8 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 451 kB 42.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 514.5 

In [107]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import spacy
from tqdm.auto import tqdm
import numpy as np
from collections import Counter

# Загрузка данных

In [39]:
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt
!wget https://raw.githubusercontent.com/aaaksenova/NLP_ABSA_project/change/data/dev_pred_aspects.txt

--2021-12-26 19:55:18--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57508 (56K) [text/plain]
Saving to: ‘dev_aspects.txt.1’

dev_aspects.txt.1   100%[===================>]  56.16K  --.-KB/s    in 0.001s  

2021-12-26 19:55:18 (63.4 MB/s) - ‘dev_aspects.txt.1’ saved [57508/57508]

--2021-12-26 19:55:18--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 20

In [40]:
dev_gold = pd.read_csv('dev_aspects.txt', sep='\t', names=['id', 'category', 'aspect', 'start', 'end', 'sentiment'])
dev_cats = pd.read_csv('dev_pred_aspects.txt', sep='\t', names=['id', 'category', 'aspect', 'start', 'end', 'sentiment'])
dev_reviews = pd.read_csv('dev_reviews.txt', sep='\t', names=['id', 'text'])

In [41]:
dev_cats

,id,category,aspect,start,end,sentiment
0,13823,Whole,аппетит,7,16,NaN
1,13823,Service,встретил,138,146,NaN
2,13823,Service,менеджер,147,155,NaN
3,13823,Service,девушка,179,186,NaN
4,13823,Service,проводила к столу,188,205,NaN
...,...,...,...,...,...,...
1115,11770,Food,стейк,831,836,NaN
1116,11770,Food,блюдо тартар с сырой рыбой и сырым яйцом,896,936,NaN
1117,11770,Service,Официанты,938,947,NaN
1118,11770,Service,обстановкая,976,987,NaN


In [42]:
dev_reviews

,id,text
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ..."
1,1427,Здравствуйте!Посетили ваше заведение вчера пер...
2,16714,"Были в пятницу (19.03.10), заказывали столик д..."
3,797,"Были в ресторане 2 раза. Один раз днем, все по..."
4,34710,Удивляюсь отзывам про хорошее обслуживание. Бы...
...,...,...
66,9216,Вы брали этот ресторан так как он близко от до...
67,8996,"Были с друзьями в пабе Метрополь, всё очень по..."
68,38299,"Случайно увидели акцию на сайте купонов, решил..."
69,37819,Очень долго выбирали ресторан на Новогодний ка...


##  Словарь тональных слов

In [7]:
!wget https://raw.githubusercontent.com/dkulagin/kartaslov/master/dataset/kartaslovsent/kartaslovsent.csv

--2021-12-26 18:57:14--  https://raw.githubusercontent.com/dkulagin/kartaslov/master/dataset/kartaslovsent/kartaslovsent.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2655438 (2.5M) [text/plain]
Saving to: ‘kartaslovsent.csv’

kartaslovsent.csv   100%[===================>]   2.53M  --.-KB/s    in 0.06s   

2021-12-26 18:57:15 (39.0 MB/s) - ‘kartaslovsent.csv’ saved [2655438/2655438]



In [8]:
tonal_words = pd.read_csv('kartaslovsent.csv', sep=';')

In [9]:
tonal_words

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19
...,...,...,...,...,...,...,...,...
46122,ёмкость,NEUT,0.00,0.167,0.000,0.690,0.143,0.00
46123,ёрзать,NGTV,-0.54,0.050,0.446,0.397,0.107,0.00
46124,ёрничать,NGTV,-0.79,0.078,0.529,0.236,0.157,0.00
46125,ёрш,NEUT,0.16,0.224,0.072,0.576,0.128,0.00


In [10]:
neg = tonal_words[tonal_words.tag == 'NGTV'].term.tolist()
pos = tonal_words[tonal_words.tag == 'PSTV'].term.tolist()

In [11]:
pos += ['спасибо'] # почему бы и не добавить

# Sentiment Extraction

In [12]:
nlp = spacy.load('ru_core_news_lg')

In [13]:
def tokenize(sent):
    return ' '.join([t.text for t in nlp(sent)])
                    

def get_token_ids(sentence, spans):
    """
    extract aspect term token ids from preprocessed text
    """
    for i, span in enumerate(spans):
        start = (0 if i == 0 else spans[i-1]['end'])
        if len(span['token'].split()) > 1:
            # euristic:
            # take root of the modified sentence
            # will not affect verb phrases but will affect compound nouns:
            # 'девушка хостес' -> root = 'хостес'
            # 'красивая девушка хостес' -> root = 'девушка'
            potential_root = [
                (t.text, t.pos_) for t in nlp('красивый ' + span['token']) 
                if t.dep_ == 'ROOT'
                ][0]
            root_token = (
                potential_root[0] if potential_root[1] == 'NOUN' else
                [t.text for t in nlp(span['token']) if t.dep_ == 'ROOT'][0]
                )
        else:
            root_token = span['token']
        span['root'] = root_token
        before = sentence[start:span['end']]
        before = tokenize(before).split(root_token)[0]
        token_id = len(before.split())
        token_id += len(tokenize(sentence[:start]).split())
        span['token_id'] = token_id
    return spans

In [14]:
def extract_sentiment(sentence, aspects, pos, neg):
    sent_dict = dict()
    aspect_ids = [i['token_id'] for i in aspects]
    aspect_terms = [i['root'] for i in aspects]
    transp = {i['token_id']: i['token'] for i in aspects}
    sentence = nlp(sentence)
    opinion_words = neg + pos
    debug = 0
    negated = False
    for token in sentence:
        # check if the word is an opinion word, then assign sentiment
        if token.lemma_ in opinion_words:
            sentiment = 1 if token.lemma_ in pos else -1
            # if target is an adverb modifier
            # but happens to be an opinion word, ignore and pass
            if token.dep_ == "advmod":
                continue
            elif token.dep_ == "amod" and token.head.i in aspect_ids:
                sent_dict[token.head.i] = sentiment
            # for opinion words that are adjectives, adverbs, verbs...
            else:
                for child in token.children:
                    # This could be better updated for modifiers that either positively or negatively emphasize
                    # check for negation words and flip the sign of sentiment
                    if child.text == "не":
                        sentiment *= -1
                        negated = True
                for child in token.children:
                    if (child.dep_ == "dobj" or child.dep_ == "nsubj"):      
                        sent_dict[child.i] = sentiment
                        # check for conjugates (a AND b), then add both to dictionary
                        subchildren = []
                        conj = 0
                        for subchild in child.children:
                            if subchild.text == "и":
                                conj=1
                            if (conj == 1) and (subchild.text != "и"):
                                subchildren.append(subchild)
                                conj = 0
                            elif subchild.text in aspect_terms:
                                subchildren.append(subchild)
                        for subchild in subchildren:
                            if subchild.i not in sent_dict:
                                sent_dict[subchild.i] = sentiment

                # additional check for "bad" annotation labels
                for child in token.children:
                    if child.i in aspect_ids and child.i not in sent_dict:
                        sent_dict[child.i] = sentiment

                # check for negation
                for child in token.head.children:
                    if (child.text == "не") and not negated: 
                        sentiment *= -1

                for child in token.head.children:
                    if (child.pos_ == "NOUN") and child.i in aspect_ids and child.i not in sent_dict:
                        sent_dict[child.i] = sentiment
                        subchildren = []
                        conj = 0
                        for subchild in child.children:
                            if subchild.text == "и":
                                conj=1
                            if (conj == 1) and (subchild.text != "и"):
                                subchildren.append(subchild)
                                conj = 0
                            elif subchild.text in aspect_terms:
                                subchildren.append(subchild)
                        for subchild in subchildren:
                            if subchild.text not in sent_dict:
                                sent_dict[subchild.i] = sentiment

                    debug += 1
    
    aspects = list(zip(aspect_terms, aspect_ids))
    sent_dict = [(transp[term[1]], sent_dict[term[1]]) if term[1] in sent_dict else (transp[term[1]], 0) for term in aspects]
    return sent_dict

In [54]:
def absa(df_reviews, df_aspects):
    df_aspects['sentiment'] = None
    le = {0: 'neutral', 1: 'positive', -1: 'negative'}

    for ind in tqdm(df_reviews.id.unique()):
        sent = df_reviews[df_reviews.id==ind]

        # get aspect 
        aspects = df_aspects[df_aspects.id==ind][['aspect', 'start', 'end']].values
        aspects = [{'token': x[0], 'start': x[1], 'end': x[2]} for x in aspects]
        aspects = get_token_ids(sent['text'].values[0], aspects)

        pred = extract_sentiment(sent.text.values[0], aspects, pos, neg)
        pred = [le[i[1]] for i in pred]
        df_aspects.loc[df_aspects.id==ind, 'sentiment'] = pred

In [55]:
absa(dev_reviews, dev_cats)

  0%|          | 0/71 [00:00<?, ?it/s]

In [106]:
dev_cats

,id,category,aspect,start,end,sentiment
0,13823,Whole,аппетит,7,16,neutral
1,13823,Service,встретил,138,146,neutral
2,13823,Service,менеджер,147,155,positive
3,13823,Service,девушка,179,186,neutral
4,13823,Service,проводила к столу,188,205,neutral
...,...,...,...,...,...,...
1115,11770,Food,стейк,831,836,neutral
1116,11770,Food,блюдо тартар с сырой рыбой и сырым яйцом,896,936,positive
1117,11770,Service,Официанты,938,947,positive
1118,11770,Service,обстановкая,976,987,positive


In [105]:
dev_cats.to_csv('dev_pred_aspects_tds.txt', sep='\t', header=False)

In [87]:
CATEGORIES = ['Whole', 'Interior', 'Service', 'Food', 'Price']

In [98]:
def get_full_sentiment(df):
    out = []
    for ind in df.id.unique():
        text_categories = df[df.id==ind][['category', 'sentiment']].groupby('category')
        asp_counter = {cat: Counter(vals['sentiment'].values) for cat, vals in text_categories}
        for c in CATEGORIES:
            if c not in asp_counter:
                s = 'absence'
            elif len(asp_counter[c]) == 1:
                s = asp_counter[c].most_common(1)[0][0]
            else:
                s = 'both'
            out.append([ind, c, s])
    return pd.DataFrame(out, columns=['id', 'category', 'sentiment'])

In [101]:
text_sents = get_full_sentiment(dev_cats)

In [103]:
text_sents

,id,category,sentiment
0,13823,Whole,both
1,13823,Interior,absence
2,13823,Service,both
3,13823,Food,positive
4,13823,Price,both
...,...,...,...
350,11770,Whole,both
351,11770,Interior,absence
352,11770,Service,positive
353,11770,Food,both


In [108]:
text_sents.to_csv('dev_pred_cats_tds.txt', sep='\t')